**https://github.com/bojone/SPACES**

In [ ]:
# ! git clone https://github.com/bojone/SPACES.git
! git clone https://github.com/dlxj/SPACES.git

Cloning into 'SPACES'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 119 (delta 52), reused 90 (delta 27), pack-reused 0
Receiving objects: 100% (119/119), 30.19 MiB | 7.95 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [ ]:
! unzip ./SPACES/datasets.zip -d ./SPACES

Archive:  ./SPACES/datasets.zip
   creating: ./SPACES/datasets/
  inflating: ./SPACES/datasets/README.md  
  inflating: ./SPACES/datasets/train_origin.json  
  inflating: ./SPACES/datasets/train_tiny.json  
  inflating: ./SPACES/datasets/user_dict.txt  
  inflating: ./SPACES/datasets/user_dict_2.txt  


In [ ]:
! python ./SPACES/split_dataset.py

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
! gdown --id '1jMAKIJmPn7kADgD3yQZhpsqM-IRM1qZt' #--output data.zip
! gdown --id '1k-dz9n-HPzUC_X1SPOB9HD2m9pV_g5Fv'
! unzip ./chinese_roberta_wwm_ext_L-12_H-768_A-12.zip -d ./chinese_roberta_wwm_ext_L-12_H-768_A-12
! unzip ./NEZHA-Base.zip

Mounted at /content/gdrive
Downloading...
From: https://drive.google.com/uc?id=1jMAKIJmPn7kADgD3yQZhpsqM-IRM1qZt
To: /content/chinese_roberta_wwm_ext_L-12_H-768_A-12.zip
382MB [00:03, 99.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1k-dz9n-HPzUC_X1SPOB9HD2m9pV_g5Fv
To: /content/NEZHA-Base.zip
1.12GB [00:13, 83.1MB/s]
Archive:  ./chinese_roberta_wwm_ext_L-12_H-768_A-12.zip
  inflating: ./chinese_roberta_wwm_ext_L-12_H-768_A-12/bert_config.json  
  inflating: ./chinese_roberta_wwm_ext_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: ./chinese_roberta_wwm_ext_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: ./chinese_roberta_wwm_ext_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: ./chinese_roberta_wwm_ext_L-12_H-768_A-12/vocab.txt  
Archive:  ./NEZHA-Base.zip
   creating: NEZHA-Base/
  inflating: NEZHA-Base/bert_config.json  
  inflating: NEZHA-Base/model.ckpt-900000.data-00000-of-00001  
  inflating: NEZHA-Base/model.ckpt-900000.index  
  inflating

In [ ]:
! pip install bert4keras==0.9.7
! pip install keras==2.3.1
! pip install rouge jieba

     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 378kB 19.6MB/s 
     |████████████████████████████████| 51kB 9.2MB/s 
  Created wheel for bert4keras: filename=bert4keras-0.9.7-cp37-none-any.whl size=43299 sha256=600bf7247abde5ddda1dff850dd326e44501b7eb5c24ba57434d37d2bd06251d
  Stored in directory: /root/.cache/pip/wheels/71/7b/06/4b4bca2005cfccd3a157cb012d1f91a83c252442c9358c238c
Successfully built bert4keras
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [ ]:
import os
#os.environ["TF_KERAS"] = '1'  # 关键配置，出错的时侯切换试试
%tensorflow_version 1.14

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [ ]:
#! -*- coding: utf-8 -*-
# 法研杯2020 司法摘要
# 工具代码合集
# 注：最好写绝对路径，否则可能出现无法预料的错误。
# 科学空间：https://kexue.fm

import numpy as np
from rouge import Rouge
import os, sys
import jieba
from bert4keras.snippets import open

# 自定义词典
user_dict_path = './SPACES/datasets/user_dict.txt'
user_dict_path_2 = './SPACES/datasets/user_dict_2.txt'
jieba.load_userdict(user_dict_path)
jieba.initialize()

# 设置递归深度
sys.setrecursionlimit(1000000)

# 标注数据
data_json = './SPACES/datasets/train.json'

# 保存权重的文件夹
if not os.path.exists('weights'):
    os.mkdir('weights')

# bert配置
config_path = './chinese_roberta_wwm_ext_L-12_H-768_A-12/bert_config.json' # './BERT/chinese_roberta_wwm_ext_L-12_H-768_A-12/bert_config.json'
checkpoint_path = './chinese_roberta_wwm_ext_L-12_H-768_A-12/bert_model.ckpt'
dict_path = './chinese_roberta_wwm_ext_L-12_H-768_A-12/vocab.txt'

# nezha配置
nezha_config_path = './NEZHA-Base/bert_config.json'
nezha_checkpoint_path = './NEZHA-Base/model.ckpt-900000'
nezha_dict_path = './NEZHA-Base/vocab.txt'

# 将数据划分N份，一份作为验证集
# import pdb; pdb.set_trace();
num_folds = 15

# 指标名
metric_keys = ['main', 'rouge-1', 'rouge-2', 'rouge-l']

# 计算rouge用
rouge = Rouge()


def load_user_dict(filename):
    """加载用户词典
    """
    user_dict = []
    with open(filename, encoding='utf-8') as f:
        for l in f:
            w = l.split()[0]
            user_dict.append(w)
    return user_dict


def data_split(data, fold, num_folds, mode):
    """划分训练集和验证集
    """
    if mode == 'train':
        D = [d for i, d in enumerate(data) if i % num_folds != fold]
    else:
        D = [d for i, d in enumerate(data) if i % num_folds == fold]

    if isinstance(data, np.ndarray):
        return np.array(D)
    else:
        return D


def compute_rouge(source, target, unit='word'):
    """计算rouge-1、rouge-2、rouge-l
    """
    if unit == 'word':
        source = jieba.cut(source, HMM=False)
        target = jieba.cut(target, HMM=False)
    source, target = ' '.join(source), ' '.join(target)
    try:
        scores = rouge.get_scores(hyps=source, refs=target)
        return {
            'rouge-1': scores[0]['rouge-1']['f'],
            'rouge-2': scores[0]['rouge-2']['f'],
            'rouge-l': scores[0]['rouge-l']['f'],
        }
    except ValueError:
        return {
            'rouge-1': 0.0,
            'rouge-2': 0.0,
            'rouge-l': 0.0,
        }


def compute_metrics(source, target, unit='word'):
    """计算所有metrics
    """
    metrics = compute_rouge(source, target, unit)
    metrics['main'] = (
        metrics['rouge-1'] * 0.2 + metrics['rouge-2'] * 0.4 +
        metrics['rouge-l'] * 0.4
    )
    return metrics


def compute_main_metric(source, target, unit='word'):
    """计算主要metric
    """
    return compute_metrics(source, target, unit)['main']


Using TensorFlow backend.
Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.811 seconds.
Prefix dict has been built successfully.


In [ ]:
#! -*- coding: utf-8 -*-
# 法研杯2020 司法摘要
# 抽取式：数据转换
# 科学空间：https://kexue.fm

import os
import json
import numpy as np
from tqdm import tqdm
from bert4keras.snippets import open
from bert4keras.snippets import text_segmentate
from bert4keras.snippets import parallel_apply
#from snippets import *

# 初始化
maxlen = 256


def text_split(text, limited=True):
    """将长句按照标点分割为多个子句。
    """
    texts = text_segmentate(text, 1, u'\n。；：，')
    if limited:
        texts = texts[-maxlen:]
    return texts


def extract_matching(texts, summaries, start_i=0, start_j=0):
    """在texts中找若干句子，使得它们连起来与summaries尽可能相似
    算法：texts和summaries都分句，然后找出summaries最长的句子，在texts
          中找与之最相似的句子作为匹配，剩下部分递归执行。
    """
    if len(texts) == 0 or len(summaries) == 0:
        return []
    i = np.argmax([len(s) for s in summaries])
    j = np.argmax([compute_main_metric(t, summaries[i], 'char') for t in texts])
    lm = extract_matching(texts[:j + 1], summaries[:i], start_i, start_j)
    rm = extract_matching(
        texts[j:], summaries[i + 1:], start_i + i + 1, start_j + j
    )
    return lm + [(start_i + i, start_j + j)] + rm


def extract_flow(inputs):
    """单个样本的构建流（给parallel_apply用）
    """
    text, summary = inputs
    texts = text_split(text, True)  # 取后maxlen句
    summaries = text_split(summary, False)
    mapping = extract_matching(texts, summaries)
    labels = sorted(set([i[1] for i in mapping]))
    pred_summary = ''.join([texts[i] for i in labels])
    metric = compute_main_metric(pred_summary, summary)
    return texts, labels, summary, metric


def load_data(filename):
    """加载数据
    返回：[(text, summary)]
    """
    D = []
    with open(filename, encoding='utf-8') as f:
        for l in f:
            l = json.loads(l)
            text = '\n'.join([d['sentence'] for d in l['text']])
            D.append((text, l['summary']))
    return D


def convert(data):
    """分句，并转换为抽取式摘要
    """
    D = parallel_apply(
        func=extract_flow,
        iterable=tqdm(data, desc=u'转换数据'),
        workers=100,
        max_queue_size=200
    )
    total_metric = sum([d[3] for d in D])
    D = [d[:3] for d in D]
    print(u'抽取结果的平均指标: %s' % (total_metric / len(D)))
    return D


if __name__ == '__main__':

    data_random_order_json = data_json[:-5] + '_random_order.json'
    data_extract_json = data_json[:-5] + '_extract.json'

    data = load_data(data_json)
    data = convert(data)

    if os.path.exists(data_random_order_json):
        idxs = json.load(open(data_random_order_json))
    else:
        idxs = list(range(len(data)))
        np.random.shuffle(idxs)
        json.dump(idxs, open(data_random_order_json, 'w'))

    data = [data[i] for i in idxs]

    with open(data_extract_json, 'w', encoding='utf-8') as f:
        for d in data:
            d = repr(d)
            d = eval(d)
            f.write(json.dumps(d, ensure_ascii=False) + '\n')

    print(u'输入数据：%s' % data_json)
    print(u'数据顺序：%s' % data_random_order_json)
    print(u'输出路径：%s' % data_extract_json)

转换数据: 100%|██████████| 1000/1000 [02:45<00:00,  6.03it/s]


抽取结果的平均指标: 0.6326052740062496
输入数据：./SPACES/datasets/train.json
数据顺序：./SPACES/datasets/train_random_order.json
输出路径：./SPACES/datasets/train_extract.json


In [ ]:
#! -*- coding: utf-8 -*-
# 法研杯2020 司法摘要
# 抽取式：句向量化
# 科学空间：https://kexue.fm

import json
import numpy as np
from tqdm import tqdm
from bert4keras.backend import keras, K
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.snippets import sequence_padding
from keras.models import Model


class GlobalAveragePooling1D(keras.layers.GlobalAveragePooling1D):
    """自定义全局池化
    """
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = K.cast(mask, K.floatx())[:, :, None]
            return K.sum(inputs * mask, axis=1) / K.sum(mask, axis=1)
        else:
            return K.mean(inputs, axis=1)


# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

# 加载bert模型，补充平均池化
encoder = build_transformer_model(
    config_path,
    checkpoint_path,
)
output = GlobalAveragePooling1D()(encoder.output)
encoder = Model(encoder.inputs, output)


def load_data(filename):
    """加载数据
    返回：[texts]
    """
    D = []
    with open(filename) as f:
        for l in f:
            texts = json.loads(l)[0]
            D.append(texts)
    return D


def vectorize_predict(texts):
    """句子列表转换为句向量
    """
    batch_token_ids, batch_segment_ids = [], []
    for text in texts:
        token_ids, segment_ids = tokenizer.encode(text, maxlen=512)
        batch_token_ids.append(token_ids)
        batch_segment_ids.append(segment_ids)
    batch_token_ids = sequence_padding(batch_token_ids)
    batch_segment_ids = sequence_padding(batch_segment_ids)
    outputs = encoder.predict([batch_token_ids, batch_segment_ids])
    return outputs


def convert(data):
    """转换所有样本
    """
    embeddings = []
    for texts in tqdm(data, desc=u'向量化'):
        outputs = vectorize_predict(texts)
        embeddings.append(outputs)
    embeddings = sequence_padding(embeddings)
    return embeddings


if __name__ == '__main__':

    data_extract_json = data_json[:-5] + '_extract.json'
    data_extract_npy = data_json[:-5] + '_extract'

    data = load_data(data_extract_json)
    embeddings = convert(data)
    np.save(data_extract_npy, embeddings)
    print(u'输出路径：%s.npy' % data_extract_npy)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


向量化:   0%|          | 0/1000 [00:00<?, ?it/s]

向量化: 100%|██████████| 1000/1000 [08:25<00:00,  1.98it/s]


输出路径：./SPACES/datasets/train_extract.npy


In [ ]:
#! -*- coding: utf-8 -*-
# 法研杯2020 司法摘要
# 抽取式：主要模型
# 科学空间：https://kexue.fm

import json
import numpy as np
from tqdm import tqdm
from bert4keras.backend import keras, K
from bert4keras.layers import LayerNormalization
from bert4keras.optimizers import Adam
from bert4keras.snippets import open
from keras.layers import *
from keras.models import Model

# 配置信息
input_size = 768
hidden_size = 384
epochs = 20
batch_size = 64
threshold = 0.2
data_extract_json = data_json[:-5] + '_extract.json'
data_extract_npy = data_json[:-5] + '_extract.npy'

if len(sys.argv) == 1:
    fold = 0
else:
    fold = 0 # int(sys.argv[1])


def load_data(filename):
    """加载数据
    返回：[(texts, labels, summary)]
    """
    D = []
    with open(filename, encoding='utf-8') as f:
        for l in f:
            D.append(json.loads(l))
    return D


class ResidualGatedConv1D(Layer):
    """门控卷积
    """
    def __init__(self, filters, kernel_size, dilation_rate=1, **kwargs):
        super(ResidualGatedConv1D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.dilation_rate = dilation_rate
        self.supports_masking = True

    def build(self, input_shape):
        super(ResidualGatedConv1D, self).build(input_shape)
        self.conv1d = Conv1D(
            filters=self.filters * 2,
            kernel_size=self.kernel_size,
            dilation_rate=self.dilation_rate,
            padding='same',
        )
        self.layernorm = LayerNormalization()

        if self.filters != input_shape[-1]:
            self.dense = Dense(self.filters, use_bias=False)

        self.alpha = self.add_weight(
            name='alpha', shape=[1], initializer='zeros'
        )

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            inputs = inputs * mask[:, :, None]

        outputs = self.conv1d(inputs)
        gate = K.sigmoid(outputs[..., self.filters:])
        outputs = outputs[..., :self.filters] * gate
        outputs = self.layernorm(outputs)

        if hasattr(self, 'dense'):
            inputs = self.dense(inputs)

        return inputs + self.alpha * outputs

    def compute_output_shape(self, input_shape):
        shape = self.conv1d.compute_output_shape(input_shape)
        return (shape[0], shape[1], shape[2] // 2)

    def get_config(self):
        config = {
            'filters': self.filters,
            'kernel_size': self.kernel_size,
            'dilation_rate': self.dilation_rate
        }
        base_config = super(ResidualGatedConv1D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


x_in = Input(shape=(None, input_size))
x = x_in

x = Masking()(x)
x = Dropout(0.1)(x)
x = Dense(hidden_size, use_bias=False)(x)
x = Dropout(0.1)(x)
x = ResidualGatedConv1D(hidden_size, 3, dilation_rate=1)(x)
x = Dropout(0.1)(x)
x = ResidualGatedConv1D(hidden_size, 3, dilation_rate=2)(x)
x = Dropout(0.1)(x)
x = ResidualGatedConv1D(hidden_size, 3, dilation_rate=4)(x)
x = Dropout(0.1)(x)
x = ResidualGatedConv1D(hidden_size, 3, dilation_rate=8)(x)
x = Dropout(0.1)(x)
x = ResidualGatedConv1D(hidden_size, 3, dilation_rate=1)(x)
x = Dropout(0.1)(x)
x = ResidualGatedConv1D(hidden_size, 3, dilation_rate=1)(x)
x = Dropout(0.1)(x)
x = Dense(1, activation='sigmoid')(x)

extract_model = Model(x_in, x)
extract_model.compile(
    loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy']
)
extract_model.summary()


def evaluate(data, data_x, threshold=0.2):
    """验证集评估
    """
    y_pred = extract_model.predict(data_x)[:, :, 0]
    total_metrics = {k: 0.0 for k in metric_keys}
    for d, yp in tqdm(zip(data, y_pred), desc=u'评估中'):
        yp = yp[:len(d[0])]
        yp = np.where(yp > threshold)[0]
        pred_summary = ''.join([d[0][i] for i in yp])
        metrics = compute_metrics(pred_summary, d[2], 'char')
        for k, v in metrics.items():
            total_metrics[k] += v
    return {k: v / len(data) for k, v in total_metrics.items()}


class Evaluator(keras.callbacks.Callback):
    """训练回调
    """
    def __init__(self):
        self.best_metric = 0.0

    def on_epoch_end(self, epoch, logs=None):
        metrics = evaluate(valid_data, valid_x, threshold + 0.1)
        if metrics['main'] >= self.best_metric:  # 保存最优 
            self.best_metric = metrics['main']
            extract_model.save_weights('weights/extract_model.%s.weights' % fold)
        metrics['best'] = self.best_metric
        print(metrics)


if __name__ == '__main__':

    # 加载数据
    data = load_data(data_extract_json)
    data_x = np.load(data_extract_npy)
    data_y = np.zeros_like(data_x[..., :1])

    for i, d in enumerate(data):
        for j in d[1]:
            data_y[i, j] = 1

    
    for i in range(15):
      fold = i

      train_data = data_split(data, fold, num_folds, 'train')
      valid_data = data_split(data, fold, num_folds, 'valid')
      train_x = data_split(data_x, fold, num_folds, 'train')
      valid_x = data_split(data_x, fold, num_folds, 'valid')
      train_y = data_split(data_y, fold, num_folds, 'train')
      valid_y = data_split(data_y, fold, num_folds, 'valid')

      # 启动训练
      evaluator = Evaluator()

      extract_model.fit(
        train_x,
        train_y,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[evaluator]
      )

else:

    extract_model.load_weights('weights/extract_model.%s.weights' % fold)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 768)         0         
_________________________________________________________________
masking_1 (Masking)          (None, None, 768)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 768)         0         
_________________________________________________________________
dense_73 (Dense)             (None, None, 384)         294912    
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 384)         0         
_________________________________________________________________
residual_gated_conv1d_1 (Res (None, None, 384)         886273    
_______________________________

评估中: 67it [00:03, 18.40it/s]


{'main': 0.47150361450100087, 'rouge-1': 0.5164771993768497, 'rouge-2': 0.3917375099213072, 'rouge-l': 0.5287829266427705, 'best': 0.47150361450100087}
Epoch 2/20
933/933 [==============================] - 2s 2ms/step - loss: 0.1869 - accuracy: 0.8966


评估中: 67it [00:01, 33.86it/s]


{'main': 0.44858239032952746, 'rouge-1': 0.4652723506712778, 'rouge-2': 0.3768019746936682, 'rouge-l': 0.5120178257945114, 'best': 0.47150361450100087}
Epoch 3/20
933/933 [==============================] - 2s 2ms/step - loss: 0.1685 - accuracy: 0.9086


...

{'main': 0.7439373502620567, 'rouge-1': 0.7683891402231403, 'rouge-2': 0.6711649825402951, 'rouge-l': 0.8044838230032764, 'best': 0.7441700260805362}
Epoch 19/20
934/934 [==============================] - 2s 2ms/step - loss: 0.0025 - accuracy: 0.9985


评估中: 66it [00:04, 14.78it/s]


{'main': 0.7440836288915915, 'rouge-1': 0.7688437971288793, 'rouge-2': 0.6714818913460858, 'rouge-l': 0.8043052823184531, 'best': 0.7441700260805362}
Epoch 20/20
934/934 [==============================] - 2s 2ms/step - loss: 0.0030 - accuracy: 0.9982


评估中: 66it [00:04, 14.72it/s]


{'main': 0.7441700260805362, 'rouge-1': 0.7688343097379746, 'rouge-2': 0.6715240873290766, 'rouge-l': 0.8044838230032764, 'best': 0.7441700260805362}


In [ ]:
#! -*- coding: utf-8 -*-
# 法研杯2020 司法摘要
# 生成式：数据转换
# 科学空间：https://kexue.fm

from bert4keras.snippets import open


def fold_convert(data, data_x, fold):
    """每一fold用对应的模型做数据转换
    """
    valid_data = data_split(data, fold, num_folds, 'valid')
    valid_x = data_split(data_x, fold, num_folds, 'valid')

    extract_model.load_weights('weights/extract_model.%s.weights' % fold)
    y_pred = extract_model.predict(valid_x)[:, :, 0]

    results = []
    for d, yp in tqdm(zip(valid_data, y_pred), desc=u'转换中'):
        yp = yp[:len(d[0])]
        yp = np.where(yp > threshold)[0]
        source_1 = ''.join([d[0][i] for i in yp])
        source_2 = ''.join([d[0][i] for i in d[1]])
        result = {
            'source_1': source_1,
            'source_2': source_2,
            'target': d[2],
        }
        results.append(result)

    return results


def convert(filename, data, data_x):
    """转换为生成式数据
    """
    F = open(filename, 'w', encoding='utf-8')
    total_results = []
    for fold in range(num_folds):
        total_results.append(fold_convert(data, data_x, fold))

    # 按照原始顺序写入到文件中
    n = 0
    while True:
        i, j = n % num_folds, n // num_folds
        try:
            d = total_results[i][j]
        except:
            break
        F.write(json.dumps(d, ensure_ascii=False) + '\n')
        n += 1

    F.close()


if __name__ == '__main__':

    data = load_data(data_extract_json)
    data_x = np.load(data_extract_npy)
    data_seq2seq_json = data_json[:-5] + '_seq2seq.json'
    convert(data_seq2seq_json, data, data_x)
    print(u'输出路径：%s' % data_seq2seq_json)

转换中: 67it [00:00, 38681.12it/s]
转换中: 67it [00:00, 50588.37it/s]
转换中: 67it [00:00, 55570.17it/s]
转换中: 67it [00:00, 57977.79it/s]
转换中: 67it [00:00, 50643.07it/s]
转换中: 67it [00:00, 53344.41it/s]
转换中: 67it [00:00, 57703.98it/s]
转换中: 67it [00:00, 57374.11it/s]
转换中: 67it [00:00, 58790.45it/s]
转换中: 67it [00:00, 58864.34it/s]
转换中: 66it [00:00, 59264.41it/s]
转换中: 66it [00:00, 33388.50it/s]
转换中: 66it [00:00, 37177.55it/s]
转换中: 66it [00:00, 56656.58it/s]
转换中: 66it [00:00, 57030.09it/s]

输出路径：./SPACES/datasets/train_seq2seq.json


In [ ]:
#! -*- coding: utf-8 -*-
# 法研杯2020 司法摘要
# 生成式：正式模型
# 科学空间：https://kexue.fm

import os, json
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from bert4keras.backend import keras, K, batch_gather
from bert4keras.layers import Loss
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer, load_vocab
from bert4keras.optimizers import Adam, extend_with_exponential_moving_average
from bert4keras.snippets import sequence_padding, open
from bert4keras.snippets import DataGenerator, AutoRegressiveDecoder
from bert4keras.snippets import longest_common_subsequence
from keras.layers import Input, Dense, Lambda
from keras.models import Model
#from snippets import *

# 基本参数
maxlen = 1024
batch_size = 5
epochs = 1024
k_sparse = 10
data_seq2seq_json = data_json[:-5] + '_seq2seq.json'
seq2seq_config_json = data_json[:-10] + 'seq2seq_config.json'

if len(sys.argv) == 1:
    fold = 0
else:
    fold = 0 # int(sys.argv[1])


def load_data(filename):
    """加载数据
    返回：[{...}]
    """
    D = []
    with open(filename) as f:
        for l in f:
            D.append(json.loads(l))
    return D


if os.path.exists(seq2seq_config_json):
    token_dict, keep_tokens, compound_tokens = json.load(
        open(seq2seq_config_json)
    )
else:
    # 加载并精简词表
    token_dict, keep_tokens = load_vocab(
        dict_path=nezha_dict_path,
        simplified=True,
        startswith=['[PAD]', '[UNK]', '[CLS]', '[SEP]'],
    )
    pure_tokenizer = Tokenizer(token_dict.copy(), do_lower_case=True)
    user_dict = []
    for w in load_user_dict(user_dict_path) + load_user_dict(user_dict_path_2):
        if w not in token_dict:
            token_dict[w] = len(token_dict)
            user_dict.append(w)
    compound_tokens = [pure_tokenizer.encode(w)[0][1:-1] for w in user_dict]
    json.dump([token_dict, keep_tokens, compound_tokens],
              open(seq2seq_config_json, 'w'))

tokenizer = Tokenizer(
    token_dict,
    do_lower_case=True,
    pre_tokenize=lambda s: jieba.cut(s, HMM=False)
)


def generate_copy_labels(source, target):
    """构建copy机制对应的label
    """
    mapping = longest_common_subsequence(source, target)[1]
    source_labels = [0] * len(source)
    target_labels = [0] * len(target)
    i0, j0 = -2, -2
    for i, j in mapping:
        if i == i0 + 1 and j == j0 + 1:
            source_labels[i] = 2
            target_labels[j] = 2
        else:
            source_labels[i] = 1
            target_labels[j] = 1
        i0, j0 = i, j
    return source_labels, target_labels


def random_masking(token_ids):
    """对输入进行随机mask，增加泛化能力
    """
    rands = np.random.random(len(token_ids))
    return [
        t if r > 0.15 else np.random.choice(token_ids)
        for r, t in zip(rands, token_ids)
    ]


class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids = [], []
        batch_output_ids, batch_labels = [], []
        for is_end, d in self.sample(random):
            i = np.random.choice(2) + 1 if random else 1
            source, target = d['source_%s' % i], d['target']
            token_ids, segment_ids = tokenizer.encode(
                source, target, maxlen=maxlen, pattern='S*ES*E'
            )
            idx = token_ids.index(tokenizer._token_end_id) + 1
            masked_token_ids = random_masking(token_ids)
            source_labels, target_labels = generate_copy_labels(
                masked_token_ids[:idx], token_ids[idx:]
            )
            labels = source_labels + target_labels[1:]
            batch_token_ids.append(masked_token_ids)
            batch_segment_ids.append(segment_ids)
            batch_output_ids.append(token_ids)
            batch_labels.append(labels)
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_output_ids = sequence_padding(batch_output_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [
                    batch_token_ids, batch_segment_ids, \
                    batch_output_ids, batch_labels
                ], None
                batch_token_ids, batch_segment_ids = [], []
                batch_output_ids, batch_labels = [], []


class CrossEntropy(Loss):
    """交叉熵作为loss，并mask掉输入部分
    """
    def compute_loss(self, inputs, mask=None):
        seq2seq_loss = self.compute_seq2seq_loss(inputs, mask)
        copy_loss = self.compute_copy_loss(inputs, mask)
        self.add_metric(seq2seq_loss, 'seq2seq_loss')
        self.add_metric(copy_loss, 'copy_loss')
        return seq2seq_loss + 2 * copy_loss

    def compute_seq2seq_loss(self, inputs, mask=None):
        y_true, y_mask, _, y_pred, _ = inputs
        y_true = y_true[:, 1:]  # 目标token_ids
        y_mask = y_mask[:, :-1] * y_mask[:, 1:]  # segment_ids，刚好指示了要预测的部分
        y_pred = y_pred[:, :-1]  # 预测序列，错开一位
        # 正loss
        pos_loss = batch_gather(y_pred, y_true[..., None])[..., 0]
        # 负loss
        y_pred = tf.nn.top_k(y_pred, k=k_sparse)[0]
        neg_loss = K.logsumexp(y_pred, axis=-1)
        # 总loss
        loss = neg_loss - pos_loss
        loss = K.sum(loss * y_mask) / K.sum(y_mask)
        return loss

    def compute_copy_loss(self, inputs, mask=None):
        _, y_mask, y_true, _, y_pred = inputs
        y_mask = K.cumsum(y_mask[:, ::-1], axis=1)[:, ::-1]
        y_mask = K.cast(K.greater(y_mask, 0.5), K.floatx())
        y_mask = y_mask[:, 1:]  # mask标记，减少一位
        y_pred = y_pred[:, :-1]  # 预测序列，错开一位
        loss = K.sparse_categorical_crossentropy(y_true, y_pred)
        loss = K.sum(loss * y_mask) / K.sum(y_mask)
        return loss


model = build_transformer_model(
    nezha_config_path,
    nezha_checkpoint_path,
    model='nezha',
    application='unilm',
    with_mlm='linear',
    keep_tokens=keep_tokens,  # 只保留keep_tokens中的字，精简原字表
    compound_tokens=compound_tokens,
)

output = model.get_layer('MLM-Norm').output
output = Dense(3, activation='softmax')(output)
outputs = model.outputs + [output]

# 预测用模型
model = Model(model.inputs, outputs)

# 训练用模型
y_in = Input(shape=(None,))
l_in = Input(shape=(None,))
outputs = [y_in, model.inputs[1], l_in] + outputs
outputs = CrossEntropy([3, 4])(outputs)

train_model = Model(model.inputs + [y_in, l_in], outputs)

AdamEMA = extend_with_exponential_moving_average(Adam, name='AdamEMA')
optimizer = AdamEMA(learning_rate=2e-5, ema_momentum=0.9999)
train_model.compile(optimizer=optimizer)
train_model.summary()


class AutoSummary(AutoRegressiveDecoder):
    """seq2seq解码器
    """
    def get_ngram_set(self, x, n):
        """生成ngram合集，返回结果格式是:
        {(n-1)-gram: set([n-gram的第n个字集合])}
        """
        result = {}
        for i in range(len(x) - n + 1):
            k = tuple(x[i:i + n])
            if k[:-1] not in result:
                result[k[:-1]] = set()
            result[k[:-1]].add(k[-1])
        return result

    @AutoRegressiveDecoder.wraps(default_rtype='logits', use_states=True)
    def predict(self, inputs, output_ids, states):
        token_ids, segment_ids = inputs
        token_ids = np.concatenate([token_ids, output_ids], 1)
        segment_ids = np.concatenate([segment_ids, np.ones_like(output_ids)], 1)
        prediction = self.last_token(model).predict([token_ids, segment_ids])
        # states用来缓存ngram的n值
        if states is None:
            states = [0]
        elif len(states) == 1 and len(token_ids) > 1:
            states = states * len(token_ids)
        # 根据copy标签来调整概率分布
        probas = np.zeros_like(prediction[0]) - 1000  # 最终要返回的概率分布
        for i, token_ids in enumerate(inputs[0]):
            if states[i] == 0:
                prediction[1][i, 2] *= -1  # 0不能接2
            label = prediction[1][i].argmax()  # 当前label
            if label < 2:
                states[i] = label
            else:
                states[i] += 1
            if states[i] > 0:
                ngrams = self.get_ngram_set(token_ids, states[i])
                prefix = tuple(output_ids[i, 1 - states[i]:])
                if prefix in ngrams:  # 如果确实是适合的ngram
                    candidates = ngrams[prefix]
                else:  # 没有的话就退回1gram
                    ngrams = self.get_ngram_set(token_ids, 1)
                    candidates = ngrams[tuple()]
                    states[i] = 1
                candidates = list(candidates)
                probas[i, candidates] = prediction[0][i, candidates]
            else:
                probas[i] = prediction[0][i]
            idxs = probas[i].argpartition(-k_sparse)
            probas[i, idxs[:-k_sparse]] = -1000
        return probas, states

    def generate(self, text, topk=1):
        max_c_len = maxlen - self.maxlen
        token_ids, segment_ids = tokenizer.encode(text, maxlen=max_c_len)
        output_ids = self.beam_search([token_ids, segment_ids],
                                      topk)  # 基于beam search
        return tokenizer.decode(output_ids)


autosummary = AutoSummary(
    start_id=tokenizer._token_start_id,
    end_id=tokenizer._token_end_id,
    maxlen=maxlen // 2
)


def evaluate(data, topk=1, filename=None):
    """验证集评估
    """
    if filename is not None:
        F = open(filename, 'w', encoding='utf-8')
    total_metrics = {k: 0.0 for k in metric_keys}
    for d in tqdm(data, desc=u'评估中'):
        pred_summary = autosummary.generate(d['source_1'], topk)
        metrics = compute_metrics(pred_summary, d['target'])
        for k, v in metrics.items():
            total_metrics[k] += v
        if filename is not None:
            F.write(d['target'] + '\t' + pred_summary + '\n')
            F.flush()
    if filename is not None:
        F.close()
    return {k: v / len(data) for k, v in total_metrics.items()}


class Evaluator(keras.callbacks.Callback):
    """训练回调
    """
    def on_epoch_end(self, epoch, logs=None):
        optimizer.apply_ema_weights()
        if epoch == epochs - 1:
          model.save_weights('weights/seq2seq_model.%s.weights' % epoch)  # 保存模型
          model.save_weights('gdrive/MyDrive/seq2seq_model.%s.weights' % epoch)
        optimizer.reset_old_weights()


if __name__ == '__main__':

    # 加载数据
    data = load_data(data_seq2seq_json)
    train_data = data_split(data, fold, num_folds, 'train')
    valid_data = data_split(data, fold, num_folds, 'valid')

    # 启动训练
    evaluator = Evaluator()
    train_generator = data_generator(train_data, batch_size)

    train_model.fit_generator(
        train_generator.forfit(),
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        callbacks=[evaluator]
    )

else:

    model.load_weights('weights/seq2seq_model.%s.weights' % (epochs - 1))

/tensorflow-1.15.2/python3.7/keras/engine/training_utils.py:819: UserWarning: Output cross_entropy_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to cross_entropy_1.
  'be expecting any data to be passed to {0}.'.format(name))


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     multiple             12405504    Input-Token[0][0]                
                                                                 MLM-Norm[0][0]                   
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        Input-Segment[0][0]        

In [ ]:
# https://drive.google.com/file/d/1dLuOdt-ivpaqbkkjvBUAGVTMUoPRWQcx/view?usp=sharing

# 下载训练好的模型
#! gdown --id '1dLuOdt-ivpaqbkkjvBUAGVTMUoPRWQcx'


In [ ]:

import numpy as np
epochs = 1024
model.load_weights('weights/seq2seq_model.%s.weights' % (epochs - 1))

def predict(text, topk=3):
    # 抽取
    texts = text_split(text) # convert.
    vecs = vectorize_predict(texts)
    preds = extract_model.predict(vecs[None])[0, :, 0]
    preds = np.where(preds > threshold)[0]
    summary = ''.join([texts[i] for i in preds])
    # 生成
    summary = autosummary.generate(summary, topk=topk)
    # 返回
    return summary


import json
import decimal
import datetime
import pandas as pd

class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            return float(o)
        elif isinstance(o, datetime.datetime):
            return str(o)
        super(DecimalEncoder, self).default(o)

def save_json(filename, dics):
    with open(filename, 'w', encoding='utf-8') as fp:
        json.dump(dics, fp, indent=4, cls=DecimalEncoder, ensure_ascii=False)
        fp.close()

def load_json(filename):
    with open(filename, encoding='utf-8') as fp:
        js = json.load(fp)
        fp.close()
        return js

# convert string to json 
def JsonParse(s):
    return json.loads(s, strict=False )

# convert dict to string
def JsonString(d):
    return json.dumps(d, cls=DecimalEncoder, ensure_ascii=False)


dic_text = {}

datas = []

with open('./SPACES/datasets/valid.json', 'r', encoding='utf-8') as fp:
  text = fp.read()
  text = text.strip()
  textss = text.split('\n')

  n = 0
  for s in textss:
    data = json.loads(s, strict=False )
    texts = data['text']

    tmp = []
    for d in texts:
      tmp.append( d['sentence'] )

    tmptext = "\n".join(tmp)

    #if text in dic_text:
    #  continue
    #else:
    #  dic_text[text] = True

    #print(text)
    machine_summary = predict(tmptext, topk=3)
    #print(summary)
    #print('#############################################\n\n\n\n\n')

    data['machine_summary'] = machine_summary
    data['booktext'] = tmptext

    datas.append(data)
    # print(text)
    
    n += 1
    
    print( f"{n}/{len(textss)}" )
    
    #if n >= 3:
    #  break


tmp = []
for d in datas:
  tmp.append( [ d['booktext'], d['summary'], d['machine_summary'] ] )
  #tmp.append( [ d['full_name_path'], d['booktext'], d['summary'], d['machine_summary'] ] )

#csv = pd.DataFrame(tmp,columns=['目录','原文', '人工摘要', '机器摘要'])
csv = pd.DataFrame(tmp,columns=['原文', '人工摘要', '机器摘要'])
csv.to_csv('/content/summarys.csv', index=False ,encoding="utf-8")
csv.to_csv('/content/gdrive/MyDrive/summarys.csv', index=False ,encoding="utf-8")

#print(len(datas))
#print(datas[1]['text'])
#print(datas[1]['machine_summary'])





  # print(json.loads(texts[1], strict=False ))


In [ ]:
intext = """""""

sssss = predict(intext, topk=3)

print(sssss)
